In [2]:
from mpi4py import MPI
from manapy.ast import Variable
from manapy.base.base import Struct
from manapy.ddm import Domain
from manapy.partitions import MeshPartition
from manapy.solvers.advec.tools_utils import initialisation_gaussian_2d
import numpy as np
import time
from numba import cuda

import matplotlib.pyplot as plt

from timeit import default_timer as timer
from manapy.cuda.utils import VarClass
###############
# test_time
###############

def test_time(iter, fun):
  #fun()
  start_time = timer()
  for _ in range(iter):
    fun()
  end_time = timer()
  elapsed_time = (end_time - start_time) / iter
  print(f"{elapsed_time * 1000:.5f} ms")
  #print(f"{elapsed_time * 1000000:.5f} micros")

###############
# init
###############
def init(dim, mesh_path):
  running_conf = Struct(backend="numba", signature=True, cache=True, float_precision="single")
  MeshPartition(mesh_path, dim=dim, conf=running_conf, periodic=[0,0,0])

  running_conf = Struct(backend="numba", signature=True, cache =True, float_precision="single")
  domain = Domain(dim=dim, conf=running_conf)
  ne = Variable(domain=domain)
  u  = Variable(domain=domain)
  v  = Variable(domain=domain)
  w  = Variable(domain=domain)
  
  P = Variable(domain=domain)
  Pinit = 2.0
  cells = domain.cells
  initialisation_gaussian_2d(ne.cell, u.cell, v.cell, P.cell, cells.center, Pinit)

  u.face[:] = 2.
  v.face[:] = 0.
  w.face[:] = 0.
  
  u.interpolate_facetocell()
  v.interpolate_facetocell()
  w.interpolate_facetocell()
  return (domain, ne, u, v, w, P)

In [3]:
dim = 3
mesh_file = "/home/aben-ham/Desktop/work/stage/manapy/mesh/3D/cube.msh"
domain, ne, u, v, w, P = init(dim=dim, mesh_path=mesh_file)

Reading gmsh file ...
Saving partition files ...
Number of Cells: 4573
Number of Vertices: 1140
Local domain contruction ...
---------------------------------
---------------------------------
set att: BCdirichlet
set att: BCneumann
set att: BCneumannNH
set att: cell
set att: dirichletfaces
set att: face
set att: ghost
set att: gradcellx
set att: gradcelly
set att: gradcellz
set att: gradfacex
set att: gradfacey
set att: gradfacez
set att: gradhalocellx
set att: gradhalocelly
set att: gradhalocellz
set att: halo
set att: haloghost
set att: halotosend
set att: neumannNHfaces
set att: neumannfaces
set att: node
set att: psi
set att: psihalo
compile kernel_facetocell to cuda => signature=void(float32[:], float32[:], int32[:,:], int32)
compile kernel_celltoface to cuda => signature=void(float32[:], float32[:], float32[:], float32[:], int32[:,:], int32[:], int32[:], int32[:], int32[:])
compile kernel_define_halosend to cuda => signature=void(float32[:], float32[:], int32[:])
compile kernel_

In [4]:
#PETScKrylovSolver
from manapy.solvers.ls import PETScKrylovSolver

# put this code to PETScKrylovSolver init function so bypass the import of petsc4py
"""
    # try_imports(['import petsc4py',],
    #             'cannot import petsc4py solver!')
    
    # from petsc4py import PETSc as petsc
    
    #self.petsc = petsc
    self.ksp   = None
    
    self.converged_reasons = {}                                                                                                                                                                             
    # for key, val in six.iteritems(petsc.KSP.ConvergedReason.__dict__):                                                                                                                                 
    #     if isinstance(val, int):                                                                                                                                                                       
    #         self.converged_reasons[val] = key    
"""

conf = Struct(reuse_mtx=True, scheme='diamond', verbose=False, precision="single")
L = PETScKrylovSolver(domain=domain, var=P, conf=conf)

SetUp the Linear system ...


In [5]:
# args list
#? The order is imported of the argument list

VarClass.convert_to_var_class([
    domain.nodes,
    domain.faces,
    domain.cells,
    domain.halos,
    ne,
    domain,
    ne.domain._cells,
    L,
    L.var
])

args = [
  L.domain.faces.nodeid, 
  L.domain.faces.halofid,
  L.domain.nodes.cellid,
  L.domain.nodes.halonid,
  L.domain.nodes.periodicid,
  L.domain.nodes.ghostcenter,
  L.domain.nodes.haloghostcenter,
  L.domain.nodes.oldname,
  L.var.BCdirichlet,
  L.matrixinnerfaces,
  L.domain.halofaces,
  L.var.dirichletfaces
]




---------------------------------
---------------------------------
can't get attr _nbnodes => _nbnodes
can't get attr nbnodes => _nbnodes
can't set attr for R_x => can't set attribute
can't set attr for R_y => can't set attribute
can't set attr for R_z => can't set attribute
set att: _R_x
set att: _R_y
set att: _R_z
set att: _cellid
set att: _ghostcenter
set att: _ghostfaceinfo
set att: _ghostid
set att: _haloghostcenter
set att: _haloghostfaceinfo
set att: _haloghostid
set att: _halonid
set att: _lambda_x
set att: _lambda_y
set att: _lambda_z
set att: _loctoglob
set att: _name
set att: _number
set att: _oldname
set att: _periodicid
set att: _vertex
can't set attr for cellid => can't set attribute
can't set attr for ghostcenter => can't set attribute
can't set attr for ghostfaceinfo => can't set attribute
can't set attr for ghostid => can't set attribute
can't set attr for haloghostcenter => can't set attribute
can't set attr for haloghostfaceinfo => can't set attribute
can't set attr

In [6]:
from manapy.ast.functions3d import compute_3dmatrix_size as cpu_function
from manapy.cuda.manapy.ast.cuda_functions3d import get_kernel_compute_3dmatrix_size as gpu_function

numba_fun = domain.backend.compile(cpu_function, echo=True, signature=True)
cuda_fun = gpu_function()

compile compute_3dmatrix_size to cpu => signature=(int32[:,:], int32[:], int32[:,:], int32[:,:], int32[:,:], float32[:,:,:], float32[:,:,:], int32[:], int32[:], int32[:], int32[:], int32[:])
compile device_search_element to cuda => signature=int32(int32[:], int32)
compile kernel_compute_3dmatrix_size to cuda => signature=void(int32[:,:], int32[:], int32[:,:], int32[:,:], int32[:,:], float32[:,:,:], float32[:,:,:], int32[:], int32[:], int32[:], int32[:], int32[:], uint64[:])


In [7]:
numba_fun(*args)
%timeit numba_fun(*args)

606 µs ± 41 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
cuda_fun(*args)
%timeit cuda_fun(*args)

CUDADispatcher(<function get_kernel_compute_3dmatrix_size.<locals>.kernel_compute_3dmatrix_size at 0x76a57c024940>) is called
1 => VarClass<int32, (9874, 4)>
2 => VarClass<int32, (9874,)>
3 => VarClass<int32, (1140, 45)>
4 => VarClass<int32, (1140, 2)>
5 => VarClass<int32, (1140, 2)>
6 => VarClass<float32, (1140, 7, 6)>
7 => VarClass<float32, (1140, 1, 6)>
8 => VarClass<int32, (1140,)>
9 => VarClass<int32, (0,)>
10 => VarClass<int32, (8418,)>
11 => VarClass<int32, (0,)>
12 => VarClass<int32, (0,)>


/home/aben-ham/anaconda3/envs/stage/lib/python3.8/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/aben-ham/anaconda3/envs/stage/lib/python3.8/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 32 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


670 µs ± 43 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
# verify

cpu_result = numba_fun(*args)
gpu_result = cuda_fun(*args)
print(cpu_result, gpu_result, cpu_result == gpu_result)


1641832 1641832 True
